In [1]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
import os
import shutil

load_dotenv()

def load_and_process_pdfs(data_dir: str):
    """Load PDFs from directory and split into chunks."""
    loader = DirectoryLoader(
        data_dir,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

def create_vector_store(chunks, persist_directory: str):
    """Create and persist Chroma vector store."""
    # Clear existing vector store if it exists
    if os.path.exists(persist_directory):
        print(f"Clearing existing vector store at {persist_directory}")
        shutil.rmtree(persist_directory)
    
    # Initialize HuggingFace embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2",
        model_kwargs={'device': 'cpu'}
    )
    
    # Create and persist Chroma vector store
    print("Creating new vector store...")
    vectordb = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    return vectordb

def main():
    # Define directories
    data_dir ="data"
    db_dir = "chroma_db"
    
    # Process PDFs
    print("Loading and processing PDFs...")
    chunks = load_and_process_pdfs(data_dir)
    print(f"Created {len(chunks)} chunks from PDFs")
    
    # Create vector store
    print("Creating vector store...")
    vectordb = create_vector_store(chunks, db_dir)
    print(f"Vector store created and persisted at {db_dir}")

if __name__ == "__main__":
    main()

Loading and processing PDFs...
Created 53 chunks from PDFs
Creating vector store...
Clearing existing vector store at chroma_db


d:\GitHub\deepseek-r1-experiment-notebooks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating new vector store...
Vector store created and persisted at chroma_db


In [ ]:
from smolagents import OpenAIServerModel, CodeAgent, ToolCallingAgent, HfApiModel, tool, GradioUI
from dotenv import load_dotenv
from langchain_chroma import Chroma
import os

load_dotenv()

reasoning_model_id = "deepseek-r1:7b"

def get_model(model_id):
        return OpenAIServerModel(
            model_id=model_id,
            api_base="http://localhost:11434/v1",
            api_key="ollama"
        )

# Create the reasoner for better RAG
reasoning_model = get_model(reasoning_model_id)
reasoner = CodeAgent(tools=[], model=reasoning_model, add_base_tools=False, max_steps=2)

# Initialize vector store and embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'}
)
db_dir = "chroma_db"
vectordb = Chroma(persist_directory=db_dir, embedding_function=embeddings)

@tool
def rag_with_reasoner(user_query: str) -> str:
    """
    This is a RAG tool that takes in a user query and searches for relevant content from the vector database.
    The result of the search is given to a reasoning LLM to generate a response, so what you'll get back
    from this tool is a short answer to the user's question based on RAG context.

    Args:
        user_query: The user's question to query the vector database with.
    """
    # Search for relevant documents
    docs = vectordb.similarity_search(user_query, k=3)
    
    # Combine document contents
    context = "\n\n".join(doc.page_content for doc in docs)
    
    # Create prompt with context
    prompt = f"""Based on the following context, answer the user's question. Be concise and specific.
    If there isn't sufficient information, give as your answer a better query to perform RAG with.
    
Context:
{context}

Question: {user_query}

Answer:"""
    
    # Get response from reasoning model
    response = reasoner.run(prompt, reset=False)
    return response

# Create the primary agent to direct the conversation
tool_model = get_model("llama3.2")
primary_agent = ToolCallingAgent(tools=[rag_with_reasoner], model=tool_model, add_base_tools=False, max_steps=3)

# Example prompt: Compare and contrast the services offered by RankBoost and Omni Marketing
def main():
    GradioUI(primary_agent).launch()

if __name__ == "__main__":
    main()

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://51352b61a3968886e9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ hi                                                                                                              │
│                                                                                                                 │
╰─ OpenAIServerModel - llama3.2 ──────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'rag_with_reasoner' with arguments: {'user_query': "{'type': 'string', 'description': 'The user's │
│ question to query the vector database with'}"}                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Based on the following context, answer the user's question. Be concise and specific.                            │
│     If there isn't sufficient information, give as your answer a better query to perform RAG with.              │
│                                                                                                                 │
│ Context:                                                                                                        │
│ multi-word entities are not present (such as New York).                                                         │
│ We evaluate the overall accuracy for all question types, and for each question type separately (se-             │
│ mantic, syntactic). Question is assumed to be correctly answered only if the closest word to the                │
│ vector computed using the above method is exactly the same as the correct word in the question;                 │
│ synonyms are thus counted as mistakes. This also means that reaching 100% accuracy is likely                    │
│ to be impossible, as the current models do not have any input information about word morphology.                │
│ However, we believe that usefulness of the word vectors for certain applications should be positively           │
│ correlated with this accuracy metric. Further progress can be achieved by incorporating information             │
│ about structure of words, especially for the syntactic questions.                                               │
│ 4.2 Maximization of Accuracy                                                                                    │
│ We have used a Google News corpus for training the word vectors. This corpus contains about                     │
│                                                                                                                 │
│ be expected that these applications can beneﬁt from the model architectures described in this paper.            │
│ Our ongoing work shows that the word vectors can be successfully applied to automatic extension                 │
│ of facts in Knowledge Bases, and also for veriﬁcation of correctness of existing facts. Results                 │
│ from machine translation experiments also look very promising. In the future, it would be also                  │
│ interesting to compare our techniques to Latent Relational Analysis [30] and others. We believe that            │
│ our comprehensive test set will help the research community to improve the existing techniques for              │
│ estimating the word vectors. We also expect that high quality word vectors will become an important             │
│ building block for future NLP applications.                                                                     │
│ 10                                                                                                              │
│                                                                                                                 │
│ resulting vectors can be used to answer very subtle semantic relationships between words, such as               │
│ a city and the country it belongs to, e.g. France is to Paris as Germany is to Berlin. Word vectors             │
│ with such semantic relationships could be used to improve many existing NLP applications, such                  │
│ as machine translation, information retrieval and question answering systems, and may enable other              │
│ future applications yet to be invented.                                                                         │
│ 5                                                                                                               │
│                                                       

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
Here is your code snippet:
<think>
Okay, so I need to figure out what word vectors are used for based on the given context. The text mentions that 
these vectors can help answer very subtle semantic relationships between words. For example, it gives a comparison 
like France is to Paris as Germany is to Berlin. This clearly shows that the vectors capture semantic and sometimes
structural relationships between words.

The text also talks about applications such as machine translation, information retrieval systems, and question 
answering. So these are practical uses where understanding word relationships is crucial.

Additionally, it's noted that incorporating more word structure information, like morphology, could improve 
accuracy for syntactic questions. So they're aware of the limitations related to words' endings or forms, which 
suggests that they might be considering ways to enhance the model in the future.

Putting this all together, I should mention both the semantic relationships and specific applications using these 
vectors.
</think>

The word vectors are designed to capture subtle semantic and structural relationships between words. They 
facilitate advanced NLP tasks by enabling improved machine translation, information retrieval, and question 
answering systems. Specifically, they help model relationships such as France is to Paris as Germany is to Berlin. 
Incorporating more word structure could further enhance their effectiveness for syntactic questions.
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_code>
Make sure to provide correct code blobs.

[Step 0: Duration 258.92 seconds| Input tokens: 480 | Output tokens: 246]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━